In [1]:
import numpy as np
print("a")
import pandas as pd 
print("a")
import re # regular expressions => string parsing and filtering
print("a")
import nltk # natural language toolkit => classification, tokenization, stemming, ...
print("a")
import spacy # tokenizer, tagger, parser, NER, pretrained models
print("a")
import string
print("a")
pd.options.mode.chained_assignment = None


a
a
a
a
a
a


In [2]:
import pandas as pd

traindf = pd.read_csv('data/rotten_tomatoes_movie_reviews.csv')



In [3]:
traindf.head()
y_train = pd.DataFrame(traindf['scoreSentiment'])
x_train = pd.DataFrame()
x_train['text']=traindf['reviewText'].astype(str)
display(x_train.head())
y_train.head()


,text
0,Timed to be just long enough for most youngste...
1,It doesn't matter if a movie costs 300 million...
2,The choreography is so precise and lifelike at...
3,The film's out-of-touch attempts at humor may ...
4,Its clumsy determination is endearing and some...


,scoreSentiment
0,POSITIVE
1,NEGATIVE
2,POSITIVE
3,NEGATIVE
4,POSITIVE


In [4]:
from preprocessing import preprocessed


data = preprocessed(x_train[:10000])


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\comes\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\comes\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\comes\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
display(data.head())
data.to_csv("data/x_train.csv", index=False)
y_train[:10000].to_csv("data/y_train.csv", index=False)

,text_lemmatized
0,time long enough youngster brief attention spa...
1,doesnt matter cost 300 million 300 dollar good...
2,choreography precise lifelike point might wond...
3,film outoftouch attempt humor may find hunt re...
4,clumsy determination endear sometimes wildly e...


In [5]:
data.head()

,text_lemmatized
0,
1,
2,
3,
4,


In [17]:
import numpy as np
import pandas as pd 
import re 
import nltk 
import spacy 
import string
from nltk.corpus import stopwords
from collections import Counter
from data.utils import emoticons
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def preprocessed(data):
    pd.options.mode.chained_assignment = None
    nltk.download('averaged_perceptron_tagger')
    nltk.download('punkt')
    nltk.download('wordnet')
    cnt = Counter()
    
    data["text"] = data["text"].str.lower()
    data["noemoji"] = data["text"].apply(remove_emoji)
    data["noemote"] = data["noemoji"].apply(remove_emoji)
    data["wopunct"] = data["noemote"].apply(remove_punctuation)
    data["text_wo_stop"] = data["wopunct"].apply(remove_stopwords)
    totaltext = ' '.join(data["text_wo_stop"])
    splited_text = totaltext.split(' ')
    cnt = Counter(splited_text)
    most_common = cnt.most_common()
    display(most_common)
    FREQWORDS = [w for (w, word_count) in most_common if word_count>10000]
    data["text_wo_stopfreq"] = data["text_wo_stop"].apply(lambda text: remove_freqwords(text, FREQWORDS))
    
    RAREWORDS = [w for (w, word_count) in most_common[-len(FREQWORDS):]]
    print(RAREWORDS)
    print(len(RAREWORDS))
    data["text_wo_rare"] = data["text_wo_stopfreq"].apply(lambda text: remove_freqwords(text, RAREWORDS))
    data["text_lemmatized"] = data["text_wo_rare"].apply(lemmatize_words)
    display(data.head())
    data2 = pd.DataFrame(data["text_lemmatized"])
    display(data2.head())
    
    return data2

def remove_emoticons(text):
    # You don't have to learn this regular expression, this is just an example of 
    # how tedious writting regex can be
    EMOTICONS = emoticons()
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text)



def remove_emoji(text: str) -> str:
    """
    Removes emojis from the input text

    Args: 
        text (str): The input text to remove emojis from

    Returns:
        str: A next text without emojis
    """
    # define a regular expression pattern
    emoji_pattern = re.compile("[" 
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"  # Miscellaneous symbols
                           u"\U000024C2-\U0001F251"  # Enclosed characters
                           "]+", flags=re.UNICODE)   # '+' signifies that those characters can occur once or more consecutively

    text_wo_emoji = re.sub(emoji_pattern, '',text)
    return text_wo_emoji

def remove_punctuation(text: str) -> str:
    """
    Removes punctuation characters from the input text

    Args:
        text (str): The input text from which punctuation characters will be removed

    Returns:
        str: A new string with all punctuation characters removed
    """
    PUNCT_TO_REMOVE = string.punctuation
    translation_table = str.maketrans('','',PUNCT_TO_REMOVE) 
    return text.translate(translation_table)

def remove_stopwords(text: str) -> str:
    STOPWORDS = set(stopwords.words('english'))
    """
    Removes stopwords from the input text

    Args: 
        text (str): The input text from which stopwords will be removed

    Returns:
        str: A new string without stopwords
    """
    split = text.split()
    filtered_words =  [x for x in split if x not in STOPWORDS]# list comprehension, will go through split list and add the word to a new list only if the word is not in STOPWORDS
    filtered_string = " ".join(filtered_words)
    return filtered_string

def remove_freqwords(text: str, freq_words: list) -> str:
    """
    Removes a selection of frequent words from the input string

    Inputs:
        text (str): The input text from which frequent words will be removed
        freq_words (list): A list of frequent words to remove from the text

    Returns:
        str: A new string with all frequent words removed
    """
    split = text.split()
    filtered_words =  [x for x in split if x not in freq_words]# list comprehension, will go through split list and add the word to a new list only if the word is not in STOPWORDS
    filtered_string = " ".join(filtered_words)
    return filtered_string

def remove_rarewords(text: str, rare_words: list) -> str:
    """
    Removes a selection of rare words from the input string

    Inputs:
        text (str): The input text from which rare words will be removed
        rare_words (list): A list of rare words to remove from the text

    Returns:
        str: A new string with all rare words removed
    """
    # Your code here:
    # Filter out the most frequent words from a text string
    split = text.split()
    filtered_words =  [x for x in split if x not in rare_words]# list comprehension, will go through split list and add the word to a new list only if the word is not in STOPWORDS
    filtered_string = " ".join(filtered_words)
    return filtered_string


def lemmatize_words(text: str) -> str:
    
    """
    Apply lemmatization to the input string, considering words' POS tags.

    This function lemmatizes words in the input string based on their POS (Part-of-Speech) tags.
    
    Args:
        text (str): The input text to be lemmatized.

    Returns:
        str: A new string with lemmatized words.
    """
    wordnet_map = {
        'N': wordnet.NOUN,
        'V': wordnet.VERB,
        'R': wordnet.ADV,
        'J': wordnet.ADJ
    }

    pos_tagged_text = nltk.pos_tag(nltk.tokenize.word_tokenize(text))
    lemmatized_words = [lemmatizer.lemmatize(words, pos = wordnet_map.get(pos[0], wordnet.NOUN)) for words, pos in pos_tagged_text ]
    lemmatized_text = ' '.join(lemmatized_words)
    return lemmatized_text

In [18]:
test = preprocessed(x_train[:10000])

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\comes\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\comes\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\comes\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


[('film', 1577),
 ('movie', 913),
 ('one', 716),
 ('like', 648),
 ('nan', 569),
 ('story', 519),
 ('much', 397),
 ('even', 396),
 ('comedy', 375),
 ('full', 364),
 ('good', 358),
 ('films', 353),
 ('review', 305),
 ('way', 283),
 ('life', 281),
 ('time', 280),
 ('best', 277),
 ('characters', 276),
 ('spanish', 273),
 ('never', 273),
 ('make', 271),
 ('little', 271),
 ('world', 266),
 ('enough', 261),
 ('love', 252),
 ('great', 248),
 ('funny', 243),
 ('makes', 238),
 ('work', 231),
 ('us', 229),
 ('doesnt', 227),
 ('may', 227),
 ('well', 222),
 ('theres', 220),
 ('feels', 216),
 ('really', 214),
 ('drama', 212),
 ('new', 211),
 ('director', 210),
 ('performance', 206),
 ('movies', 205),
 ('still', 205),
 ('performances', 200),
 ('fun', 194),
 ('many', 194),
 ('first', 193),
 ('might', 188),
 ('thats', 187),
 ('also', 187),
 ('something', 186),
 ('cast', 185),
 ('end', 184),
 ('see', 183),
 ('character', 182),
 ('made', 182),
 ('better', 181),
 ('two', 180),
 ('documentary', 175),
 ('ma

['film', 'movie', 'one', 'like', 'nan', 'story', 'much', 'even', 'comedy', 'full', 'good', 'films', 'review', 'way', 'life', 'time', 'best', 'characters', 'spanish', 'never', 'make', 'little', 'world', 'enough', 'love', 'great', 'funny', 'makes', 'work', 'us', 'doesnt', 'may', 'well', 'theres', 'feels', 'really', 'drama', 'new', 'director', 'performance', 'movies', 'still', 'performances', 'fun', 'many', 'first', 'might', 'thats', 'also', 'something', 'cast', 'end', 'see', 'character', 'made', 'better', 'two', 'documentary', 'man', 'action', 'entertaining', 'people', 'black', 'could', 'isnt', 'would', 'though', 'get', 'look', 'without', 'thriller', 'plot', 'horror', 'yet', 'moments', 'sense', 'real', 'every', 'almost', 'long', 'another', 'script', 'jurassic', 'take', 'lot', 'kind', 'feel', 'adam', 'years', 'family', 'hard', 'far', 'tale', 'actors', 'watch', 'gets', 'quite', 'right', 'less', 'often', 'audience', 'despite', 'dont', 'go', 'interesting', 'original', 'times', 'nothing', 'sa

,text,noemoji,noemote,wopunct,text_wo_stop,text_wo_stopfreq,text_wo_rare,text_lemmatized
0,timed to be just long enough for most youngste...,timed to be just long enough for most youngste...,timed to be just long enough for most youngste...,timed to be just long enough for most youngste...,timed long enough youngsters brief attention s...,timed long enough youngsters brief attention s...,,
1,it doesn't matter if a movie costs 300 million...,it doesn't matter if a movie costs 300 million...,it doesn't matter if a movie costs 300 million...,it doesnt matter if a movie costs 300 million ...,doesnt matter movie costs 300 million 300 doll...,doesnt matter movie costs 300 million 300 doll...,,
2,the choreography is so precise and lifelike at...,the choreography is so precise and lifelike at...,the choreography is so precise and lifelike at...,the choreography is so precise and lifelike at...,choreography precise lifelike points one might...,choreography precise lifelike points one might...,,
3,the film's out-of-touch attempts at humor may ...,the film's out-of-touch attempts at humor may ...,the film's out-of-touch attempts at humor may ...,the films outoftouch attempts at humor may fin...,films outoftouch attempts humor may find hunti...,films outoftouch attempts humor may find hunti...,,
4,its clumsy determination is endearing and some...,its clumsy determination is endearing and some...,its clumsy determination is endearing and some...,its clumsy determination is endearing and some...,clumsy determination endearing sometimes wildl...,clumsy determination endearing sometimes wildl...,,


,text_lemmatized
0,
1,
2,
3,
4,
